### Тестовое задание для ШИФТ25. Data Engineerig course ###



Подгружаем используемые библиотеки для работы с данными и http get-запросами

In [11]:
import pandas as pd
import requests
import json

Делаем запрос к серверу с параметрами для получения json-объекта и дальнейшего парсинга значений по ключам, и создания на их основе словара data

In [12]:
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 55.0344,
    "longitude": 82.9434,
    "daily": "sunrise,sunset,daylight_duration",
    "hourly": "temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,"
              "temperature_80m,temperature_120m,wind_speed_10m,wind_speed_80m,wind_direction_10m,"
              "wind_direction_80m,visibility,evapotranspiration,weather_code,soil_temperature_0cm,"
              "soil_temperature_6cm,rain,showers,snowfall",
    "timezone": "auto",
    "timeformat": "unixtime",
    "wind_speed_unit": "kn",
    "temperature_unit": "fahrenheit",
    "precipitation_unit": "inch",
    "start_date": "2025-05-16",
    "end_date": "2025-05-30"
}

response = requests.get(url, params=params)
data = response.json()

Проверяем список ключей в словаре data

In [13]:

data.keys()


dict_keys(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation', 'elevation', 'hourly_units', 'hourly', 'daily_units', 'daily'])

Преобразование почасовых данных из словаря data в таблицу Pandas dataftrame 'hourly_df'

In [14]:
hourly = data['hourly']
hourly_df = pd.DataFrame(hourly)

Преобразуем время из unix-формата в привычный вид

In [15]:
hourly_df['time'] = pd.to_datetime(hourly_df['time'], unit='s')


Проверяем, что таблица Pandas 'hourly' сформировалась верно

In [30]:
hourly_df.head()

,time,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,temperature_80m,temperature_120m,wind_speed_10m,wind_speed_80m,wind_direction_10m,...,soil_temperature_0cm_c,soil_temperature_6cm_c,dew_point_2m_c,wind_speed_10m_mps,wind_speed_80m_mps,rain_mm,showers_mm,snowfall_mm,date,is_daylight
0,2025-05-15 17:00:00,48.4,56,33.4,40.9,49.2,49.2,8.0,14.4,94,...,NaN,NaN,0.777778,4.115552,7.407994,0.0,0.0,0.0,2025-05-15,False
1,2025-05-15 18:00:00,48.4,58,34.3,41.5,48.8,48.9,7.0,13.1,109,...,NaN,NaN,1.277778,3.601108,6.739216,0.0,0.0,0.0,2025-05-15,False
2,2025-05-15 19:00:00,48.6,57,34.0,41.6,48.5,48.5,7.0,12.7,104,...,NaN,NaN,1.111111,3.601108,6.533439,0.0,0.0,0.0,2025-05-15,False
3,2025-05-15 20:00:00,48.0,56,33.1,39.5,48.1,47.8,10.0,15.9,122,...,NaN,NaN,0.611111,5.144440,8.179660,0.0,0.0,0.0,2025-05-15,False
4,2025-05-15 21:00:00,47.8,56,32.9,40.2,47.4,47.2,8.1,13.9,107,...,NaN,NaN,0.500000,4.166996,7.150772,0.0,0.0,0.0,2025-05-15,False


Создаем методы/функции для конвертации величин в таблице

In [31]:
def F_to_C(f):
    return (f - 32) * 5 / 9

def inch_to_mm(inch):
    return inch * 25.4

def kn_to_mps(kn):
    return kn * 0.514444

Преобразуем величины в столбцах

In [32]:
hourly_df['temperature_2m_c'] = F_to_C(hourly_df['temperature_2m'])
hourly_df['apparent_temperature_c'] = F_to_C(hourly_df['apparent_temperature'])
hourly_df['temperature_80m_c'] = F_to_C(hourly_df['temperature_80m'])
hourly_df['temperature_120m_c'] = F_to_C(hourly_df['temperature_120m'])
hourly_df['soil_temperature_0cm_c'] = F_to_C(hourly_df['soil_temperature_0cm'])
hourly_df['soil_temperature_6cm_c'] = F_to_C(hourly_df['soil_temperature_6cm'])
hourly_df['dew_point_2m_c'] = F_to_C(hourly_df['dew_point_2m'])

hourly_df['wind_speed_10m_mps'] = kn_to_mps(hourly_df['wind_speed_10m'])
hourly_df['wind_speed_80m_mps'] = kn_to_mps(hourly_df['wind_speed_80m'])

hourly_df['rain_mm'] = inch_to_mm(hourly_df['rain'])
hourly_df['showers_mm'] = inch_to_mm(hourly_df['showers'])
hourly_df['snowfall_mm'] = inch_to_mm(hourly_df['snowfall'])


Создаем колонку 'date' для 24-часовой группировки

In [33]:
hourly_df['date'] = hourly_df['time'].dt.date

Агрегацируем данные за 24 часа в новую таблицу 'daily_avg_df' c группировкой по дате с усреднеными значениями

In [34]:
daily_avg_df = hourly_df.groupby('date').agg({
    'temperature_2m_c': 'mean',
    'relative_humidity_2m': 'mean',
    'dew_point_2m': 'mean',
    'apparent_temperature_c': 'mean',
    'temperature_80m_c': 'mean',
    'temperature_120m_c': 'mean',
    'wind_speed_10m_mps': 'mean',
    'wind_speed_80m_mps': 'mean',
    'visibility': 'mean',
    'rain_mm': 'sum',
    'showers_mm': 'sum',
    'snowfall_mm': 'sum',
}).reset_index()

Переименовываем колонки в таблице согласно ТЗ из задания

In [36]:
daily_avg_df.columns = [
    'date',
    'avg_temperature_2m_24h',
    'avg_relative_humidity_2m_24h',
    'avg_dew_point_2m_24h',
    'avg_apparent_temperature_24h',
    'avg_temperature_80m_24h',
    'avg_temperature_120m_24h',
    'avg_wind_speed_10m_24h',
    'avg_wind_speed_80m_24h',
    'avg_visibility_24h',
    'total_rain_24h',
    'total_showers_24h',
    'total_snowfall_24h'
]


Создаем таблицу с дневными значениями daily

In [37]:
daily = pd.DataFrame(data['daily'])
daily['date'] = pd.to_datetime(daily['time'], unit='s').dt.date
daily['sunrise_dt'] = pd.to_datetime(daily['sunrise'], unit='s')
daily['sunset_dt'] = pd.to_datetime(daily['sunset'], unit='s')

Создаем метод для фильтрации таблицы hourly_df по времени светового дня

In [38]:
def is_daylight(row):
    day = daily[daily['date'] == row['date']]
    if not day.empty:
        return day['sunrise_dt'].values[0] <= row['time'] <= day['sunset_dt'].values[0]
    return False

hourly_df['is_daylight'] = hourly_df.apply(is_daylight, axis=1)

Создаем таблицы 'dayligt_df' и 'gaylight_avg_df' с группировкой по дате cо средними значениямт

In [39]:
daylight_df = hourly_df[hourly_df['is_daylight']]
daylight_avg_df = daylight_df.groupby('date').agg({
    'temperature_2m_c': 'mean',
    'relative_humidity_2m': 'mean',
    'dew_point_2m': 'mean',
    'apparent_temperature_c': 'mean',
    'temperature_80m_c': 'mean',
    'temperature_120m_c': 'mean',
    'wind_speed_10m_mps': 'mean',
    'wind_speed_80m_mps': 'mean',
    'visibility': 'mean',
    'rain_mm': 'sum',
    'showers_mm': 'sum',
    'snowfall_mm': 'sum',
}).reset_index()




Переименовываем колонки в таблице согласно ТЗ из задания

In [40]:
daylight_avg_df.columns = [
    'date',
    'avg_temperature_2m_daylight',
    'avg_relative_humidity_2m_daylight',
    'avg_dew_point_2m_daylight',
    'avg_apparent_temperature_daylight',
    'avg_temperature_80m_daylight',
    'avg_temperature_120m_daylight',
    'avg_wind_speed_10m_daylight',
    'avg_wind_speed_80m_daylight',
    'avg_visibility_daylight',
    'total_rain_daylight',
    'total_showers_daylight',
    'total_snowfall_daylight'
]

Проверяем корректность таблицы 'daylight_avg_df'

In [41]:
daylight_avg_df.head()

,date,avg_temperature_2m_daylight,avg_relative_humidity_2m_daylight,avg_dew_point_2m_daylight,avg_apparent_temperature_daylight,avg_temperature_80m_daylight,avg_temperature_120m_daylight,avg_wind_speed_10m_daylight,avg_wind_speed_80m_daylight,avg_visibility_daylight,total_rain_daylight,total_showers_daylight,total_snowfall_daylight
0,2025-05-15,8.388889,59.0,33.5,4.500000,8.166667,8.055556,3.652552,6.173328,79199.477,0.0,0.0,0.0
1,2025-05-16,8.777778,88.0,44.5,7.444444,9.777778,9.888889,1.080332,1.646221,79199.477,0.0,0.0,0.0
2,2025-05-17,11.222222,95.0,50.8,10.722222,11.500000,11.444444,1.234666,2.675109,79199.477,0.0,0.0,0.0
3,2025-05-18,8.888889,87.0,44.3,5.333333,8.555556,8.388889,5.298773,8.179660,79199.477,0.0,0.0,0.0
4,2025-05-19,11.111111,72.0,43.2,8.111111,11.277778,11.277778,3.858330,6.327661,79199.477,0.0,0.0,0.0


Объединяем все данные в 1 таблицу final_df

In [42]:
final_df = pd.merge(daily_avg_df, daylight_avg_df, on='date', how='left')

Проверяем корректность итоговой таблицы'final_df'

In [43]:
final_df.head()

,date,avg_temperature_2m_24h,avg_relative_humidity_2m_24h,avg_dew_point_2m_24h,avg_apparent_temperature_24h,avg_temperature_80m_24h,avg_temperature_120m_24h,avg_wind_speed_10m_24h,avg_wind_speed_80m_24h,avg_visibility_24h,...,avg_dew_point_2m_daylight,avg_apparent_temperature_daylight,avg_temperature_80m_daylight,avg_temperature_120m_daylight,avg_wind_speed_10m_daylight,avg_wind_speed_80m_daylight,avg_visibility_daylight,total_rain_daylight,total_showers_daylight,total_snowfall_daylight
0,2025-05-15,8.857143,57.142857,33.500000,4.746032,8.849206,8.793651,4.027362,6.974391,79199.477000,...,33.5,4.500000,8.166667,8.055556,3.652552,6.173328,79199.477,0.0,0.0,0.0
1,2025-05-16,9.250000,81.875000,43.083333,6.696759,9.173611,9.120370,3.110243,4.668579,71710.959708,...,44.5,7.444444,9.777778,9.888889,1.080332,1.646221,79199.477,0.0,0.0,0.0
2,2025-05-17,14.793981,73.333333,49.512500,13.400463,14.212963,14.094907,2.760849,3.984797,78934.275500,...,50.8,10.722222,11.500000,11.444444,1.234666,2.675109,79199.477,0.0,0.0,0.0
3,2025-05-18,12.701389,84.500000,50.095833,10.407407,12.189815,12.027778,4.458515,6.818527,66313.977333,...,44.3,5.333333,8.555556,8.388889,5.298773,8.179660,79199.477,0.0,0.0,0.0
4,2025-05-19,12.312500,62.875000,41.104167,9.062500,11.939815,11.856481,4.467089,6.711351,79199.477000,...,43.2,8.111111,11.277778,11.277778,3.858330,6.327661,79199.477,0.0,0.0,0.0


Экспортируем таблицу 'final_df' в файл csv

In [29]:
final_df.to_csv("final_df_SHIFT25.csv", index=False)